# Analysis for Differential Jet Mass in Z+Jets events with NanoAODv9

In [1]:
import awkward as ak
import numpy as np
import time
import coffea
import uproot
import hist
import vector
print("awkward version ", ak.__version__)
print("coffea version ", coffea.__version__)
from coffea import util, processor
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema
from collections import defaultdict
import pickle

awkward version  1.8.0
coffea version  0.7.18


### Automatically reload any modules used

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from smp_utils import *
from response_maker_nanov9_lib import *
from response_maker_nanov9 import *

In [4]:
from dask.distributed import Client

client = Client("tls://rappoccio-40gmail-2ecom.dask.cmsaf-prod.flatiron.hollandhpc.org:8786")
client

<Client: 'tls://192.168.150.55:8786' processes=1 threads=2, memory=7.00 GiB>

### If desired, uncomment these to move or remove old pkl files

In [ ]:
#! mv qjetmass_zjets_gen.pkl qjetmass_zjets_gen_old.pkl
#! mv qjetmass_zjets_reco.pkl qjetmass_zjets_reco_old.pkl

In [ ]:
f = uproot.open('root://xcache//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-400to600_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/00846A4F-433D-6D4E-8381-DD588DAE4954.root')

### If you want to rerun, here is the line. Otherwise keep it commented to just read the pkl file. 

In [6]:

response_maker_nanov9(testing=False, do_gen=True, client=client)

FileNotFoundError: file not found ([ERROR] Server responded with an error: [3010] Unable to open /store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-400to600_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/00846A4F-433D-6D4E-8381-DD588DAE4954.root; permission denied
)

    'root://xcache//store/mc/RunIISummer20UL18NanoAODv9/DYJetsToLL_M-50_HT-400to600_TuneCP5_PSweights_13TeV-madgraphMLM-pythia8/NANOAODSIM/106X_upgrade2018_realistic_v16_L1v1-v1/130000/00846A4F-433D-6D4E-8381-DD588DAE4954.root'

Files may be specified as:
   * str/bytes: relative or absolute filesystem path or URL, without any colons
         other than Windows drive letter or URL schema.
         Examples: "rel/file.root", "C:\abs\file.root", "http://where/what.root"
   * str/bytes: same with an object-within-ROOT path, separated by a colon.
         Example: "rel/file.root:tdirectory/ttree"
   * pathlib.Path: always interpreted as a filesystem path or URL only (n

### Open pkl file for the output histograms

In [ ]:
with open("qjetmass_zjets_gen.pkl", "rb") as f:
    output = pickle.load( f )

In [ ]:
output.keys()

In [ ]:
output["cutflow"]

In [ ]:
datasets = list( output["cutflow"].keys() )

In [ ]:
histnames = output.keys()

In [ ]:
print(histnames)

### Normalize all of the histograms to the sum of the number of events

In [ ]:
for ihist,ihistname in enumerate(histnames):
    if ihistname == "cutflow" : continue
    print(ihist)
    print(ihistname)
    for i,dataset in enumerate(datasets) :
        print(i)
        print(dataset)
        if np.sum(output[ihistname].values(flow=True)) > 0:
            output[ihistname].view(flow=True)[i] /= output["cutflow"][dataset]['all events']

# Make plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
output['ptjet_mjet_u_gen'].project("pt").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_u_gen'].project("pt").plot()
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_u_gen'].project("mreco").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_g_gen'].project("mreco").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
response_matrix_u_values = output['response_matrix_u'].project("ptreco", "mreco", "ptgen", "mgen").values()
response_matrix_g_values = output['response_matrix_g'].project("ptreco", "mreco", "ptgen", "mgen").values()

In [ ]:
nptreco,nmassreco,nptgen,nmassgen = response_matrix_u_values.shape
response_matrix_u_final = response_matrix_u_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )

In [ ]:
plt.imshow( np.log(response_matrix_u_final+1), vmax=1e-3, aspect="equal", cmap="binary" )

In [ ]:
response_matrix_g_final = response_matrix_g_values.reshape( (nptreco)*(nmassreco), (nptgen)*(nmassgen) )

In [ ]:
plt.imshow( np.log(response_matrix_g_final+1), vmax=1e-3, aspect="equal", cmap="binary" )

In [ ]:
output['ptjet_gen_pre'].project("pt").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_u_gen'].project("pt").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_u_reco'].project("pt").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_u_reco'].project("mreco").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_mjet_g_reco'].project("mreco").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptjet_reco_over_gen'].project("frac").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['drjet_reco_gen'].project("dr").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['mz_gen'].project("mass").plot(density=True)
#plt.yscale("log")
plt.show()

In [ ]:
output['mz_reco'].project("mass").plot(density=True)
#plt.yscale("log")
plt.show()

In [ ]:
output['mz_reco_over_gen'].project("frac").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['dphi_z_jet_gen'].project("dphi").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptasym_z_jet_gen'].project("frac").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['ptasym_z_jet_reco'].project("frac").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['dr_z_jet_gen'].project("dr").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['dr_z_jet_reco'].project("dr").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['dphi_z_jet_reco'].project("dphi").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['m_u_jet_reco_over_gen'].project("frac").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['m_g_jet_reco_over_gen'].project("frac").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['drjet_reco_gen'].project("dr").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
output['dr_gen_subjet'].project("dr").plot(density=True)
plt.yscale("log")
plt.show()

In [ ]:
resolutions_u = output['m_u_jet_reco_over_gen'].project("ptgen", "mgen", "frac")
resolutions_g = output['m_g_jet_reco_over_gen'].project("ptgen", "mgen", "frac")

In [ ]:
nptbins,nmassbins,nfracbins = resolutions_u.shape

In [ ]:
ptvals = resolutions_u.axes['ptgen']
mvals = resolutions_u.axes['mgen']

In [ ]:
for i in range(nptbins):
    for j in range(nmassbins): 
        resolutions_g[i,j,:].plot()
        plt.title("Groomed, pt = %6.2f, m = %6.2f" % (ptvals.value(i), mvals.value(j)) )
        plt.show()

In [ ]:
for i in range(nptbins):
    for j in range(nmassbins): 
        resolutions_u[i,j,:].plot()
        plt.title("Ungroomed, pt = %6.2f, m = %6.2f" % (ptvals.value(i), mvals.value(j)) )
        plt.show()